Bayesian Optimization & NN Opt
---

In [1]:
from bayes_opt import BayesianOptimization
from nnopt.nnopt import Optimizer
import time
import numpy as np
import gym


In [2]:
ll = gym.make("MountainCarContinuous-v0")
print(ll.action_space)
print(ll.observation_space)

def black_box_functionA(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    transform = np.array([[a, b]])
    
    sc = 0
    for _ in range(4):
        s = ll.reset()
        done = False
        while not done:
            #ll.render()
            s, r, done, _ = ll.step(transform @ s)
            sc += r

    return sc / 4

def black_box_functionB(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -a ** 2 - (b - 1) ** 2 + 1

#print(black_box_function(-221.59328, 28.267517))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Box(1,)
Box(2,)


/usr/local/lib/python3.6/site-packages/gym/envs/registration.py:14: DeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
Nopt = Optimizer(black_box_functionB, N=2, surrogate_hidden_layer=1000, model_hidden_layer=400, memory_hidden_layer=400,
                 Rn=[(-10, 10), (-10, 10)]) #(-10, 10), (-10, 10), (-10, 10), (-10, 10)])

Bopt = BayesianOptimization(
    f=black_box_functionB,
    pbounds={'a': (-10, 10), 'b': (-10, 10)#, 'c': (-10, 10), 
             #'d': (-10, 10), 'e': (-10, 10), 'f': (-10, 10)},
            },
    random_state=np.random.randint(10),
)

Bayesian
---

In [4]:
timestamp = time.time()
Bopt.maximize(
    init_points=40,
    n_iter=5,
)

print("Bayesian took: {}".format(time.time() - timestamp))

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        | -95.82    | -9.793    |  0.03749  |
|  2        | -68.29    | -0.08453  | -7.323    |
|  3        | -94.18    | -7.158    | -5.629    |
|  4        | -38.11    | -1.63     | -5.038    |
|  5        | -84.93    | -8.319    | -3.09     |
|  6        | -86.6     | -6.664    |  7.571    |
|  7        | -184.9    |  9.019    | -9.225    |
|  8        | -15.06    |  3.982    |  1.455    |
|  9        | -67.83    |  7.96     |  3.338    |
|  10       | -9.209    |  0.9568   |  4.049    |
|  11       | -12.5     | -2.27     |  3.889    |
|  12       | -44.06    |  6.497    | -0.6866   |
|  13       | -102.4    |  8.81     |  6.075    |
|  14       | -150.3    |  9.667    | -6.607    |
|  15       | -61.75    |  0.4217   |  8.91     |
|  16       | -46.59    |  2.991    |  7.217    |
|  17       | -140.8    |  9.417    | -6.289    |
|  18       | -67.23    | -5.138    |  7.468    |


In [5]:
timestamp = time.time()
best, target = Nopt.run(random=50, optimization=10, fitting=10000, verbose=True)

print("Best", best, "target", target)
print(black_box_functionB(*best))
print("NNopt took:", time.time() - timestamp)

sample [-3.453188 -7.786429] target -88.12584027965045
sample [-1.764986  1.19281 ] target -2.1523514337641245
sample [1.211832 7.468994] target -42.31642210038626
sample [ 4.5980816 -7.566061 ] target -93.51975569336355
sample [-6.018343 -3.42381 ] target -54.79054708823696
sample [-5.958774  2.528017] target -36.84182474038994
sample [-2.8668118  8.254908 ] target -59.852294024320145
sample [6.2287445 8.971569 ] target -101.34317143018416
sample [ 1.4253979 -1.7844982] target -8.785189403926779
sample [-4.876044   2.0138435] target -23.80368182202278
sample [1.9563313 3.4092064] target -8.63150740311903
sample [-8.016357    0.99087715] target -63.26206954161535
sample [1.374157 2.645731] target -3.5967377616216254
sample [-4.9391723  2.9758434] target -27.29937995028581
sample [ 9.557644  -4.5438313] target -121.0826227550499
sample [4.0764446 9.342176 ] target -85.2093085000015
sample [-2.9518247  3.9146023] target -16.208175301930396
sample [ 7.6179657 -7.6945353] target -132.62834